In [103]:
import pandas as pd
import numpy as np
import os

In [104]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [105]:
df = pd.read_csv('backend\\data\\predictions\\risk.csv')
df['date'] = pd.to_datetime(df['date'])
df['pick_odds'] = df['pick_odds'].apply(lambda x: 50 * round(x / 50))
df['pick_diff'] = df['pick_diff'].apply(lambda x: .03 * round(x / .03))

In [106]:
def risk_management(diff, odds):
    if 0 <= diff < .03:
        return .1 if odds > 0 else .2
    elif .03 <= diff < .06:
        return .25 if odds > 0 else .5
    elif .06 <= diff < .09:
        return .5 if odds > 0 else 1
    elif .09 <= diff < .12:
        return .75 if odds > 0 else 1.5
    elif .12 <= diff < .15:
        return 1 if odds > 0 else 2
    elif .15 <= diff < .18:
        return 1.25 if odds > 0 else 2.5
    elif .18 <= diff < .21:
        return 1.5 if odds > 0 else 3
    elif .21 <= diff < .24:
        return 1.75 if odds > 0 else 3.5
    elif .24 <= diff < .27:
        return 2 if odds > 0 else 3.5
    elif .27 <= diff < .30:
        return 2.25 if odds < 0 else 3.5
    elif .30 <= diff:
        return 3.5 
    else:
        return None

In [107]:
df['risk_unit'] = df.apply(lambda x: risk_management(x.pick_diff, x.pick_odds), axis=1)
df['risk_profit'] = np.where(df['risk_correct'], df['potential'] * df['risk_unit'], -1 * df['risk_unit'])

In [108]:
# Per diff
per_diff = df.groupby(['pick_diff', 'pick_fav']).agg({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_diff['accuracy'] = round(per_diff[('risk_correct', 'sum')] / per_diff[('risk_correct', 'count')] * 100)
per_diff

risk_profit risk_correct       accuracy
                           sum          sum count         
pick_diff pick_fav                                        
0.00      0           0.105000            1     1    100.0
          1           0.781265           17    20     85.0
0.03      0          -0.462500            1     4     25.0
          1           0.002472           28    39     72.0
0.06      0          -0.500000            0     1      0.0
          1          -0.323195           26    38     68.0
0.09      0          -1.912500            2     7     29.0
          1          -2.623907           17    27     63.0
0.12      0          -3.150000            3    10     30.0
          1         -14.238768           11    23     48.0
0.15      0           3.937500            8    16     50.0
          1           5.508217           13    18     72.0
0.18      0           0.900000            5    11     45.0
          1           3.645065            8    12     67.0
0.21      0         -22.750000            0    13      0.0
          1           4.617797            5     7     71.0
0.24      0          -4.300000            2     7     29.0
          1           6.756619            4     5     80.0
0.27      0          18.200000            4     6     67.0
          1          -6.750000            0     3      0.0
0.30      0          14.980000            3     5     60.0
0.33      0          19.180000            4     9     44.0
0.36      0           0.700000            1     3     33.0
0.39      0           4.375000            1     1    100.0
0.42      0           2.975000            1     2     50.0
0.45      0          -3.500000            0     1      0.0

In [109]:
round(df['risk_correct'].sum() / df['risk_correct'].count() * 100)

57

In [110]:
df['risk_profit'].sum()

26.15306433703116